In [1]:
%matplotlib inline
from effective_quadratures.PolyParams import PolynomialParam
from effective_quadratures.PolyParentFile import PolyParent
from effective_quadratures.IndexSets import IndexSet
from effective_quadratures.Utils import column, evalfunction
import effective_quadratures.MatrixRoutines as mat
import effective_quadratures.ComputeStats as stats
from effective_quadratures.EffectiveQuadSubsampling import EffectiveSubsampling
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

<h1> Vegetation UQ with Effective-Quadratures

Consider a uniform variation in the following parameters:
    
$$\begin{array}{lll} \hline
Variable & Range & Description \\ \hline
\rho_{shoot} & [38.2, 250.4] & Shoot \; density \; (/m^2)\\ 
L & [0.157, 0.313] & Piston \; blade \; length \; (m)\\ 
d & [0.002, 0.01] & Diameter \;  (m)\\ 
t & [0.0002, 0.001] & Thickness \; (m)\\  \hline
\end{array}$$

Being by setting up the ranges for the four parameters

In [2]:
quadratic_pts = 3
x1 = PolynomialParam("Uniform", 38.2, 250.4, [], [], [], 3)
x2 = PolynomialParam("Uniform", 0.157, 0.313, [], [], [], 3)
x3 = PolynomialParam("Uniform", 0.002, 0.01, [], [], [], 3)
x4 = PolynomialParam("Uniform", 0.0002, 0.001, [], [], [], 3)
orders = [2,2,2,2]
x1x2x3x4 = [x1, x2, x3, x4]

Set the polynomial basis; this will dictate how many function evaluations we will require. Bear in mind that a quadratic in 4 dimensions requires $3^4=81$ points. To reduce the cost we will opt for a total order basis.

In [3]:
polybasis = IndexSet("total order", orders)
print IndexSet.getIndexSet(polybasis)
maximum_number_of_evals = IndexSet.getCardinality(polybasis)
print maximum_number_of_evals

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  2.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  1.]
 [ 0.  0.  2.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  1.]
 [ 0.  1.  1.  0.]
 [ 0.  2.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  1.  0.]
 [ 1.  1.  0.  0.]
 [ 2.  0.  0.  0.]]
15


In [4]:
effectiveQuads = EffectiveSubsampling(x1x2x3x4, polybasis, 0)
A, esquad_pts, W, indices = EffectiveSubsampling.getAsubsampled(effectiveQuads, maximum_number_of_evals)
print esquad_pts

[[  1.44300000e+02   2.35000000e-01   6.00000000e-03   6.00000000e-04]
 [  6.21152934e+01   1.74581460e-01   2.90161332e-03   6.00000000e-04]
 [  2.26484707e+02   2.95418540e-01   6.00000000e-03   2.90161332e-04]
 [  1.44300000e+02   1.74581460e-01   9.09838668e-03   9.09838668e-04]
 [  1.44300000e+02   2.95418540e-01   2.90161332e-03   9.09838668e-04]
 [  6.21152934e+01   2.95418540e-01   6.00000000e-03   2.90161332e-04]
 [  2.26484707e+02   1.74581460e-01   6.00000000e-03   2.90161332e-04]
 [  1.44300000e+02   2.95418540e-01   9.09838668e-03   9.09838668e-04]
 [  1.44300000e+02   2.35000000e-01   9.09838668e-03   2.90161332e-04]
 [  2.26484707e+02   2.35000000e-01   6.00000000e-03   9.09838668e-04]
 [  1.44300000e+02   2.35000000e-01   2.90161332e-03   2.90161332e-04]
 [  6.21152934e+01   2.35000000e-01   6.00000000e-03   9.09838668e-04]
 [  6.21152934e+01   1.74581460e-01   6.00000000e-03   2.90161332e-04]
 [  6.21152934e+01   2.35000000e-01   9.09838668e-03   6.00000000e-04]
 [  1.

Taking the output from the text file, we have:

In [13]:
Output = [15.9881,   16.5091 ,  16.0162   ,15.9950,   16.0310  , 16.4592  , 16.0958  , 15.8507 ,  16.0757  , 15.9252  , 16.4301 ,  16.1259 ,  16.4682 ,  16.0501  , 16.2200]
Output = np.mat(Output)

In [14]:
Output = Output.T
b = W * Output
A, normalizations = mat.rowNormalize(A)
b = np.dot(normalizations, b)

In [17]:
xn = mat.solveLeastSquares(A, b)
mean, variance = stats.compute_mean_variance(xn.T, IndexSet.getIndexSet(polybasis))
Sobol = stats.compute_first_order_Sobol_indices(xn.T, IndexSet.getIndexSet(polybasis))